In [1]:
import numpy as np
import pandas as pd
import math

from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import cross_val_score, KFold, train_test_split, GridSearchCV
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.linear_model import Lasso, ElasticNet
from sklearn.kernel_ridge import KernelRidge
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
import xgboost as xgb

In [2]:
SEED = 42
np.random.seed(SEED)

TARGET = "SalePrice"
TRAIN_SET = "processed_train.csv"
TEST_SET = 'processed_test.csv'

In [3]:
SEED = 42
np.random.seed(SEED)

In [4]:
df_train = pd.read_csv(TRAIN_SET)
df_test = pd.read_csv(TEST_SET)

X_test = df_test.drop(['Id'], axis=1)
test_Id = df_test['Id']

In [5]:
X_train = df_train.drop([TARGET], axis=1)
y_train = df_train[TARGET]

In [6]:
#Validation function
n_folds =  12
# n_folds =  X_train.shape[0]

def scorer_rmse(y_true, y_pred):
    score = math.sqrt(mean_squared_error(y_true, y_pred))
    return score

scorer = make_scorer(scorer_rmse, greater_is_better=False)

def cv_rsme(model, X_train, y_train):
    kf = KFold(n_folds, shuffle=True, random_state=SEED).get_n_splits(X_train)
    rmse= -cross_val_score(model, X_train.values, y_train.values, scoring=scorer, cv=kf, n_jobs=-1)
    return rmse

In [7]:
df_scores = pd.DataFrame(columns=['rsme', 'Summary'])
df_scores.index.name = 'Model'

**Lasso regression**

In [8]:
%%time

param_grid = {'alpha': [0.0005, 0.005, 0.05, 0.01, 0.1, 1],
             }

lasso = Lasso(random_state=SEED)
grid_search = GridSearchCV(lasso, param_grid=param_grid, cv=n_folds, scoring=scorer, return_train_score=True, n_jobs=-1)
grid_search.fit(X_train, y_train)
best_params_lasso = grid_search.best_params_

lasso = Lasso(**best_params_lasso, random_state=SEED)
lasso_score = cv_rsme(lasso, X_train, y_train)
print(f"lasso rsme mean: {lasso_score.mean()} std: {lasso_score.std()}")

df_scores.loc['Lasso', 'Summary'] = (lasso.get_params().keys(), lasso.get_params().values())                   
df_scores.loc['Lasso', 'rsme'] = lasso_score.mean()

lasso rsme mean: 0.11467355775891365 std: 0.011616524307848698
Wall time: 3.75 s


**Elastic regression**

In [9]:
en = ElasticNet(alpha=0.0005, l1_ratio=.9, random_state=3)
en_score = cv_rsme(en, X_train, y_train)
print(f"lasso rsme mean: {en_score.mean()} std: {en_score.std()}")

df_scores.loc['Elastic', 'Summary'] = (en.get_params().keys(), en.get_params().values())                   
df_scores.loc['Elastic', 'rsme'] = en_score.mean()

lasso rsme mean: 0.11461614279505189 std: 0.011507961490369695


**Kernel ridge regression**

In [10]:
krr = KernelRidge(alpha=0.6, kernel='polynomial', degree=2, coef0=2.5)
krr_score = cv_rsme(krr, X_train, y_train)
print(f"lasso rsme mean: {krr_score.mean()} std: {krr_score.std()}")

df_scores.loc['KRR', 'Summary'] = (krr.get_params().keys(), krr.get_params().values())                   
df_scores.loc['KRR', 'rsme'] = krr_score.mean()

lasso rsme mean: 0.11907129116183984 std: 0.013285758368296085


**Support vector regressor**

In [11]:
%%time

# param_grid = {'C': [1, 5, 10, 20],
#              'kernel': ['poly', 'rbf', 'sigmoid'],
#               'degree': [3, 5, 10],
#              'gamma': ['scale', 'auto'],
#              }

# svr = SVR()
# grid_search = GridSearchCV(svr, param_grid=param_grid, cv=n_folds, scoring=scorer, return_train_score=True, n_jobs=-1)
# grid_search.fit(X_train, y_train)
# best_params_svr = grid_search.best_params_

best_params_svr = {'C': 1, 'degree': 3, 'gamma': 'scale', 'kernel': 'rbf'}
svr = SVR(**best_params_svr)
svr_score = cv_rsme(svr, X_train, y_train)
print(f"svr rsme mean: {svr_score.mean()} std: {svr_score.std()}")

df_scores.loc['SVR', 'Summary'] = (svr.get_params().keys(), svr.get_params().values())                   
df_scores.loc['SVR', 'rsme'] = svr_score.mean()

svr rsme mean: 0.11555427118948934 std: 0.012669757383678168
Wall time: 323 ms


**Gradient boost regressor**

In [12]:
%%time

GBoost = GradientBoostingRegressor(n_estimators=2000, learning_rate=0.05,
                                   max_features='sqrt', max_depth=4,
                                   loss='huber', random_state=SEED)
GBoost_score = cv_rsme(GBoost, X_train, y_train)
print(f"GBoost rsme mean: {GBoost_score.mean()} std: {GBoost_score.std()}")


df_scores.loc['Gboost', 'rsme'] = GBoost_score.mean()
df_scores.loc['Gboost', 'Summary'] = (GBoost.get_params().keys(), GBoost.get_params().values()) 

GBoost rsme mean: 0.11029410067198851 std: 0.014525963145810379
Wall time: 26.7 s


**Xgboost**

In [13]:
%%time 

model_xgb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =7, nthread = -1)
model_xgb_score = cv_rsme(model_xgb, X_train, y_train)
print(f"RandomForest rsme mean: {model_xgb_score.mean()} std: {model_xgb_score.std()}")

df_scores.loc['Xgb', 'rsme'] = model_xgb_score.mean()
df_scores.loc['Xgb', 'Summary'] = (model_xgb.get_params().keys(), model_xgb.get_params().values()) 

RandomForest rsme mean: 0.11277971723437331 std: 0.014583308360391104
Wall time: 16.1 s


In [14]:
# Reference
# https://www.analyticsvidhya.com/blog/2020/12/improve-predictive-model-score-stacking-regressor/
class StackingAveragedModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, base_models, meta_model, n_folds=10):
        self.base_models = base_models
        self.meta_model = meta_model
        self.n_folds = n_folds
   
    # We again fit the data on clones of the original models
    def fit(self, X, y):
        self.base_models_ = [list() for x in self.base_models]
        self.meta_model_ = clone(self.meta_model)
        kfold = KFold(n_splits=self.n_folds, shuffle=True, random_state=SEED)
        
        # Train cloned base models then create out-of-fold predictions
        # that are needed to train the cloned meta-model
        out_of_fold_predictions = np.zeros((X.shape[0], len(self.base_models)))
        for i, model in enumerate(self.base_models):
            for train_index, holdout_index in kfold.split(X, y):
                instance = clone(model)
                self.base_models_[i].append(instance)
                instance.fit(X[train_index], y[train_index])
                y_pred = instance.predict(X[holdout_index])
                out_of_fold_predictions[holdout_index, i] = y_pred
                
        # Now train the cloned  meta-model using the out-of-fold predictions as new feature
        self.meta_model_.fit(out_of_fold_predictions, y)
        return self
   
    #Do the predictions of all base models on the test data and use the averaged predictions as 
    #meta-features for the final prediction which is done by the meta-model
    def predict(self, X):
        meta_features = np.column_stack([
            np.column_stack([model.predict(X) for model in base_models]).mean(axis=1)
            for base_models in self.base_models_ ])
        return self.meta_model_.predict(meta_features)

In [15]:
%%time

stacked_averaged_models = StackingAveragedModels(base_models=(svr, en, GBoost, krr),
                                                 meta_model=lasso, n_folds=n_folds)

stacking_score = cv_rsme(stacked_averaged_models, X_train, y_train)
print(f"Stacking rsme mean: {stacking_score.mean()} std: {stacking_score.std()}")


df_scores.loc['Stacking', 'Summary'] = ("base_models = (svr, en, GBoost),\
                                                 meta_model = lasso, n_folds=n_folds")                   
df_scores.loc['Stacking', 'rsme'] = stacking_score.mean()

Stacking rsme mean: 0.10653210371641864 std: 0.012996660250046676
Wall time: 5min 7s


## Submission

In [16]:
stacked_averaged_models.fit(X_train.values, y_train.values)
preds = np.expm1(stacked_averaged_models.predict(X_test.values))

In [17]:
sub = pd.DataFrame()
sub['Id'] = test_Id
sub[TARGET] = preds
sub.to_csv('submission.csv',index=False)